# Utilities

In this part of the tutorial, we'll learn how to use some of Hemlock's utilities: figures and random assignment.

Run the cell below to create a test app.

In [ ]:
import os

from flask_login import current_user
from hemlock import User, Page, create_test_app
from hemlock.questions import Input, Label
from hemlock.utils.random import Assigner
from hemlock.utils.statics import make_figure

os.environ.pop("GITPOD_HOST", None)

app = create_test_app()

## Figures

We can add a figure to a label with the `make_figure` function. The first argument is the image source.

In this example, the image source is a URL.

In [ ]:
Label(
    make_figure(
        "https://imgs.xkcd.com/comics/wanna_see_the_code.png",
        figure_align="center"
    )
).display()

We can also add our own images. These are stored in a folder named `static` in the project's root directory (see the file explorer to the left).

**Tip.** If you're using a large number of your own images (5-10 or more), don't store them in the `static` folder. Instead, upload them to cloud storage, make sure the URLs are public, and use those URLs as the image source.

In [ ]:
Label(
    make_figure("../static/code_quality.png", figure_align="center")
).display()

**Note.** To use an imagine in your survey file, `my_survey.py`, file, replace `"../static/IMAGE_FILENAME"` with `flask.url_for("static", filename="IMAGE_FILENAME")`.

So, instead of,

```python
Label(
    make_figure("../static/code_quality.png"), figure_align="center")
)
```

use

```python
from flask import url_for

Label(
    make_figure(url_for("static", filename="code_quality.png"), figure_align="center")
)
```

## Random assignment

We can randomly assign users to conditions using the `Assigner`. The first argument is a dictionary mapping factor names to possible assignment values.

In the example below, we create an `Assigner` to randomly assign users to either a treatment or control condition.

In [ ]:
assigner = Assigner({"treatment": (0, 1)})
user = User.make_test_user()
assigner.assign_user(user)

Notice that `assign_user` returns a dictionary mapping factor names to the user's assigned values.

The user's assignment is also stored in its metadata.

In [ ]:
user.get_meta_data()["treatment"]

Here's what random assignment looks like in a seed function. Notice that we don't have to pass the `user` argument to `assign_user` in this context.

In [ ]:
assigner = Assigner({"treatment": (0, 1)})

def seed():
    assignment = assigner.assign_user()
    print("Treatment is", assignment["treatment"])
    print("Treatment is", current_user.get_meta_data()["treatment"])
    return [
        Page(
            Label("Hello, world!")
        )
    ]


_ = User.make_test_user(seed).test_get()

In [ ]:
_ = User.make_test_user(seed).test_get()

## Exercises

Image we're creating a survey with a 2-factor design. On page 0, we'll show users a vignette to put them in a happy or sad mood. On page 1, we'll ask them to rate how funny one of two cartoons is.

0. Create a survey with 3 pages:

    0. A page with a `Label` with a vignette to put the user in a happy mood.
    1. A page with an `Input` displaying a cartoon and asking the user to rate how funny it is.
    2. A goodbye page.
1. Create an assigner with 2 factors:

    0. A factor named `"mood"`. The values are `"happy"` and `"sad"`.
    1. A factor named `"image"`. The values are image sources. Use at least 1 image from a URL and 1 image from the `static` folder.

2. Assign users to conditions by calling `assigner.assign_user()` in the seed function.
3. Change the `Label` on page 0 so that, if the user was assigned to the `"sad"` condition, they see a vignette to put them in a sad mood.
4. Change page 1 so the user sees their assigned image.
5. Run test users through your survey and make sure the assigner and images are working as expected.
6. Transfer the survey you wrote in steps 0-4 to `src/my_survey.py`, run the app, and verify that it's working as expected. Make sure to change `"../static/IMAGE_FILENAME"` to `url_for("static", filename="IMAGE_FILENAME")`.

**Note.** If you're working in Gitpod, you likely won't see the image in the static folder when you run your app. This is completely fine. The image will display properly when it counts (that is, when you deploy your app and send it to users).

7. Test your code with `make test`.

In [ ]:
# WRITE YOUR CODE HERE

## Answers

In [ ]:
assigner = Assigner(
    {"image": ("code_quality", "wanna_see_the_code"), "mood": ("happy", "sad")}
)


def seed():
    assignment = assigner.assign_user()
    if assignment["image"] == "code_quality":
        img_src = "../static/code_quality.png"
    else:
        img_src = "https://imgs.xkcd.com/comics/wanna_see_the_code.png"
        
    return [
        Page(
            Label(
                f"""
                Here is a vignette designed to put you in a {assignment['mood']} mood.
                """
            )
        ),
        Page(
            Input(
                f"""
                From 1 (not funny at all) to 5 (very funny), how funny is this image?
                {make_figure(img_src, figure_align="center")}
                """,
                input_tag={"type": "number", "min": 1, "max": 5}
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]

In [ ]:
User.make_test_user(seed).test()

See `src/utils.py` for what your survey file should look like.

Now you know how to use some of Hemlock's utilities! Check out `110_deployment.md` for the next part of the tutorial.